In [34]:

import re
import pandas as pd
import bs4
import requests
import spacy
from spacy import displacy

# nlp = spacy.load('en_core_web_sm')
# nlp = spacy.load('vi_core_news_lg')
# nlp = spacy.load('vi_spacy_model')
# nlp = spacy.load("xx_sent_ud_sm")
nlp = spacy.load("xx_ent_wiki_sm")

from spacy.matcher import Matcher
from spacy.tokens import Span

import networkx as nx

import matplotlib.pyplot as plt
from tqdm import tqdm

pd.set_option('display.max_colwidth', 200)
%matplotlib inline

In [35]:
# doc = nlp("Hôm nay trời nắng to")
# displacy.serve(doc, style="dep")
# 
doc = nlp("Hôm nay trời nắng to")
# displacy.serve(doc, style="ent")
print(doc)

Hôm nay trời nắng to


In [36]:
doc = nlp('Cộng đồng xử lý ngôn ngữ tự nhiên')
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)


Cộng     Xxxx True False
đồng     xxxx True False
xử     xx True False
lý     xx True False
ngôn     xxxx True False
ngữ     xxx True False
tự     xx True False
nhiên     xxxx True False


In [37]:
# import wikipedia sentences
# candidate_sentences = pd.read_csv("wiki_sentences_v2.csv")
candidate_sentences = pd.read_csv("sentences_vn.csv")
candidate_sentences.shape

(488, 1)

In [11]:
candidate_sentences['sentence'].sample(5)

154             vào tháng 1 năm 2017, johns và berg đã báo cáo cho emmerich.
475         dàn diễn viên xuất hiện thông qua hình ba chiều ma của hạt tiêu.
426                         thiết kế sản xuất đã được xử lý bởi t. muthuraj.
9                                         chúng tôi chỉ cố gắng làm bộ phim.
396    giá trị nghệ thuật của âm nhạc điện ảnh thường xuyên được tranh luận.
Name: sentence, dtype: object

In [38]:
# doc = nlp("the drawdown process is governed by astm standard d823")
doc = nlp("bối rối và thất vọng, connie quyết định rời đi một mình.")
print(doc)

for tok in doc:
  print(tok.text, "...", tok.dep_)

bối rối và thất vọng, connie quyết định rời đi một mình.
bối ... 
rối ... 
và ... 
thất ... 
vọng ... 
, ... 
connie ... 
quyết ... 
định ... 
rời ... 
đi ... 
một ... 
mình ... 
. ... 


In [39]:
def get_entities(sent):
    ## chunk 1
    ent1 = ""
    ent2 = ""

    prv_tok_dep = ""  # dependency tag of previous token in the sentence
    prv_tok_text = ""  # previous token in the sentence

    prefix = ""
    modifier = ""

    #############################################################

    for tok in nlp(sent):
        ## chunk 2
        # nếu token là một dấu chấm câu thì chuyển sang token tiếp theo
        if tok.dep_ != "punct":
            # check: token có phải là một từ ghép hay không
            if tok.dep_ == "compound":
                prefix = tok.text
                # nếu từ trước đó cũng là một 'từ ghép' thì hãy thêm từ hiện tại vào nó
                if prv_tok_dep == "compound":
                    prefix = prv_tok_text + " " + tok.text

            # check: token là một bổ ngữ hay không
            if tok.dep_.endswith("mod") == True:
                modifier = tok.text
                # nếu từ trước đó cũng là một 'từ ghép' thì hãy thêm từ hiện tại vào nó
                if prv_tok_dep == "compound":
                    modifier = prv_tok_text + " " + tok.text

            ## chunk 3
            if tok.dep_.find("subj") == True:
                ent1 = modifier + " " + prefix + " " + tok.text
                prefix = ""
                modifier = ""
                prv_tok_dep = ""
                prv_tok_text = ""

            ## chunk 4
            if tok.dep_.find("obj") == True:
                ent2 = modifier + " " + prefix + " " + tok.text

            ## chunk 5
            # update variables
            prv_tok_dep = tok.dep_
            prv_tok_text = tok.text
    #############################################################

    return [ent1.strip(), ent2.strip()]


In [40]:
get_entities("bối rối và thất vọng, connie quyết định rời đi một mình.")

['', '']

In [41]:
entity_pairs = []

for i in tqdm(candidate_sentences["sentence"]):
    entity_pairs.append(get_entities(i))

100%|██████████| 488/488 [00:01<00:00, 334.60it/s]


In [42]:
entity_pairs[10:20]

[['', ''],
 ['', ''],
 ['', ''],
 ['', ''],
 ['', ''],
 ['', ''],
 ['', ''],
 ['', ''],
 ['', ''],
 ['', '']]

In [43]:

def get_relation(sent):

    doc = nlp(sent)

    # Matcher class object
    matcher = Matcher(nlp.vocab)
    print("matcher: ", matcher)
    print("nlp.vocab: ", nlp.vocab)

    #define the pattern
    pattern = [[{
        'DEP': 'ROOT'
    }, {
        'DEP': 'prep',
        'OP': "?"
    }, {
        'DEP': 'agent',
        'OP': "?"
    }, {
        'POS': 'ADJ',
        'OP': "?"
    }]]

    matcher.add("matching_1", patterns = pattern)

    matches = matcher(doc)
    k = len(matches) - 1  
    # print("matches: ", matches)
    # print("type(matches): ", type(matches))
    # print("len(matches): ", len(matches))
    # print("k: ", k)
    
    tmpSpan = ""
    if len(matches) >= 1:
      span = doc[matches[k][1]:matches[k][2]]
      tmpSpan = span.text
      # print("span: ", span.text)
      # print("type(span): ", type(span.text))
      # print("---")

    # return (span.text)
    return(tmpSpan)

In [44]:
# get_relation("John completed the task")
get_relation("bối rối và thất vọng, connie quyết định rời đi một mình.")

matcher:  <spacy.matcher.matcher.Matcher object at 0x7fed1100bb40>
nlp.vocab:  <spacy.vocab.Vocab object at 0x7fed0fd190d0>


ValueError: [E155] The pipeline needs to include a morphologizer or tagger+attribute_ruler in order to use Matcher or PhraseMatcher with the attribute POS. Try using `nlp()` instead of `nlp.make_doc()` or `list(nlp.pipe())` instead of `list(nlp.tokenizer.pipe())`.

In [45]:
relations = [get_relation(i) for i in tqdm(candidate_sentences['sentence'])]

  0%|          | 0/488 [00:00<?, ?it/s]

matcher:  <spacy.matcher.matcher.Matcher object at 0x7fed0f7fd940>
nlp.vocab:  <spacy.vocab.Vocab object at 0x7fed0fd190d0>


ValueError: [E155] The pipeline needs to include a morphologizer or tagger+attribute_ruler in order to use Matcher or PhraseMatcher with the attribute POS. Try using `nlp()` instead of `nlp.make_doc()` or `list(nlp.pipe())` instead of `list(nlp.tokenizer.pipe())`.

In [71]:
pd.Series(relations).value_counts()[:50]

với        15
tháng      15
có         14
nhạc       12
ảnh        12
được       10
nhất        9
hoan        8
diễn        8
tôi         7
của         6
đầu         6
kiến        5
hành        5
những       5
cầu         5
cảm         5
cùng        5
thành       4
tiên        4
án          3
nhau        3
sung        3
viên        3
giám        3
do          3
địa         3
ra          3
thương      3
kịch        3
phương      3
loạt        3
làm         3
nó          3
thật        3
cung        3
giới        3
thất        2
theo        2
chính       2
aquaman     2
giờ         2
hậu         2
lillis      2
đại         2
chương      2
lại         2
giao        2
nhận        2
cố          2
dtype: int64

In [72]:
# extract subject
source = [i[0] for i in entity_pairs]

# extract object
target = [i[1] for i in entity_pairs]

# print('source', len(source))
# print('target', len(target))
# print('edge', len(relations))

kg_df = pd.DataFrame({'source': source, 'target': target, 'edge': relations})

In [73]:
# create a directed-graph from a dataframe
G=nx.from_pandas_edgelist(kg_df, "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

In [76]:
plt.figure(figsize=(12,12))

pos = nx.spring_layout(G)
nx.draw(G, with_labels=True, node_color='skyblue', edge_cmap=plt.cm.Blues, pos = pos)
plt.show()

AttributeError: module 'scipy.sparse' has no attribute 'coo_array'

<Figure size 864x864 with 0 Axes>

In [52]:
# create a directed-graph from a dataframe
G = nx.from_pandas_edgelist(kg_df[kg_df['edge'] == "composed by"],
                            "source",
                            "target",
                            edge_attr=True,
                            create_using=nx.MultiDiGraph())

In [79]:
print(G)

MultiDiGraph with 538 nodes and 488 edges


In [80]:
plt.figure(figsize=(12, 12))
pos = nx.spring_layout(G, k=0.5)  # k regulates the distance between nodes
nx.draw(G,
        with_labels=True,
        node_color='skyblue',
        node_size=1500,
        edge_cmap=plt.cm.Blues,
        pos=pos)
plt.show()

AttributeError: module 'scipy.sparse' has no attribute 'coo_array'

<Figure size 864x864 with 0 Axes>